<a href="https://colab.research.google.com/github/CFeenan/SolarCNN/blob/master/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import torch
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.pool = nn.MaxPool1d(kernel_size=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 7, 64)  # 61 → 30 → 15 → 7 (after 3 poolings)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 1)  # Output is a single logit

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # Now shape: (batch, 128, 7)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x  # No sigmoid here — handled in BCEWithLogitsLoss


In [39]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load the .npy files for train and test data
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Output the shapes for confirmation
X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape


(torch.Size([3750, 3, 61]),
 torch.Size([3750]),
 torch.Size([750, 3, 61]),
 torch.Size([750]))

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import BCEWithLogitsLoss

# Define model
model = CNN1D()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Compute class imbalance weight
healthy, faulty = torch.bincount(y_train_tensor.long())
pos_weight = torch.tensor([2.0], dtype=torch.float32).to(device)

# Loss function with class weight
criterion = BCEWithLogitsLoss(pos_weight=pos_weight)

# Optimiser
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop
for epoch in range(30):
    model.train()
    epoch_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")


Epoch 1, Avg Loss: 0.5198
Epoch 2, Avg Loss: 0.3132
Epoch 3, Avg Loss: 0.3074
Epoch 4, Avg Loss: 0.2609
Epoch 5, Avg Loss: 0.2248
Epoch 6, Avg Loss: 0.2246
Epoch 7, Avg Loss: 0.2098
Epoch 8, Avg Loss: 0.2262
Epoch 9, Avg Loss: 0.2063
Epoch 10, Avg Loss: 0.1778
Epoch 11, Avg Loss: 0.1778
Epoch 12, Avg Loss: 0.1853
Epoch 13, Avg Loss: 0.1667
Epoch 14, Avg Loss: 0.1676
Epoch 15, Avg Loss: 0.1484
Epoch 16, Avg Loss: 0.2061
Epoch 17, Avg Loss: 0.1635
Epoch 18, Avg Loss: 0.2068
Epoch 19, Avg Loss: 0.1712
Epoch 20, Avg Loss: 0.1435
Epoch 21, Avg Loss: 0.1709
Epoch 22, Avg Loss: 0.1548
Epoch 23, Avg Loss: 0.1348
Epoch 24, Avg Loss: 0.1537
Epoch 25, Avg Loss: 0.1157
Epoch 26, Avg Loss: 0.1273
Epoch 27, Avg Loss: 0.1206
Epoch 28, Avg Loss: 0.1410
Epoch 29, Avg Loss: 0.1317
Epoch 30, Avg Loss: 0.1151


In [41]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
predictions, actuals = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs).squeeze()
        preds = (outputs > 0.45).int()
        predictions.extend(preds.tolist())
        actuals.extend(labels.int().tolist())

print(confusion_matrix(actuals, predictions))
print(classification_report(actuals, predictions, target_names=["Healthy", "Faulty"]))


[[556  14]
 [ 41 139]]
              precision    recall  f1-score   support

     Healthy       0.93      0.98      0.95       570
      Faulty       0.91      0.77      0.83       180

    accuracy                           0.93       750
   macro avg       0.92      0.87      0.89       750
weighted avg       0.93      0.93      0.92       750



In [33]:
torch.save(model.state_dict(), "cnn_model_v4.pth")
